In [1]:
# Install required libraries (Run this once in Colab)
!pip install spacy transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
# Load spaCy and BERT
import spacy
import torch
from transformers import BertTokenizer, BertModel

# Load small spaCy model (acts like Word2Vec)
print("Loading spaCy (Word2Vec-like embeddings)...")
nlp = spacy.load("en_core_web_sm")

# Load BERT model & tokenizer
print("Loading BERT model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Define sentences
sentence1 = "A bat flew in the night sky."
sentence2 = "He hit the ball with a bat."

word = "bat"  # Target word


Loading spaCy (Word2Vec-like embeddings)...
Loading BERT model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Function to get spaCy (Word2Vec-like) embeddings
def get_spacy_embedding(word):
    return nlp(word).vector

In [ ]:
# Function to get BERT embeddings
def get_bert_embedding(sentence, target_word):
    tokens = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = bert_model(**tokens)
    embeddings = outputs.last_hidden_state.squeeze(0)
    word_index = tokens.input_ids.squeeze(0).tolist().index(tokenizer.convert_tokens_to_ids(target_word))
    return embeddings[word_index].numpy()

In [3]:
# Get embeddings
spacy_emb1 = get_spacy_embedding(word)
spacy_emb2 = get_spacy_embedding(word)

bert_emb1 = get_bert_embedding(sentence1, word)
bert_emb2 = get_bert_embedding(sentence2, word)

# Compare similarities
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

In [4]:

print("\n🔹 SpaCy (Word2Vec-like) Similarity")
print(f"Similarity: {cosine_similarity(spacy_emb1, spacy_emb2):.4f} (❌ Doesn't understand context!)")

print("\n🔹 BERT (Contextual) Similarity")
print(f"Similarity: {cosine_similarity(bert_emb1, bert_emb2):.4f} (✅ Captures different meanings!)")



🔹 SpaCy (Word2Vec-like) Similarity
Similarity: 1.0000 (❌ Doesn't understand context!)

🔹 BERT (Contextual) Similarity
Similarity: 0.5355 (✅ Captures different meanings!)


In [5]:
import numpy as np
np.set_printoptions(precision=5, suppress=True)  # Format for readability

print("\n📌 SpaCy Embedding for 'bat' (Word2Vec-like):")
print(spacy_emb1)

print("\n📌 BERT Embedding for 'bat' in Sentence 1:")
print(bert_emb1)

print("\n📌 BERT Embedding for 'bat' in Sentence 2:")
print(bert_emb2)


📌 SpaCy Embedding for 'bat' (Word2Vec-like):
[-0.0031  -0.65994  0.19868  0.29042 -0.31615 -0.77254  1.37278  1.32693
 -0.6375  -0.09236  0.59921 -1.20226 -0.86889  0.11879  0.47347  0.10275
 -0.83393 -0.65337 -0.09545  0.00637 -0.26951  1.34141 -0.08993 -0.10028
  0.46462  0.69362  0.51638  0.22193 -0.01104  1.41402 -0.34573  0.44538
 -0.50134  0.27036 -0.57527 -0.9376  -0.06556  0.86533  0.05047  0.76329
 -0.45157  0.19856  0.49282  0.30799  0.10728 -0.03811 -0.11013 -0.32085
 -0.06732 -0.97787 -0.20232 -0.02787  0.24088 -0.45478  0.78604  0.00003
  1.03465 -0.47882 -0.33885  0.42913 -1.38636  0.04957  0.1821   0.24602
 -0.22062 -0.22776 -0.33387  0.01595 -0.3354   0.09905  0.40767  0.88593
  0.52786 -0.12645  0.52102 -0.08341  0.42505 -0.29436 -0.76539 -0.1278
 -0.42138  0.08081 -0.72071  0.0038   0.02326  0.28554  0.25362  0.18923
 -0.82988  0.16009 -1.31893  0.84316 -0.00615 -0.04892 -0.08445  0.31289]

📌 BERT Embedding for 'bat' in Sentence 1:
[ 0.76158 -0.00937 -0.4191  -0.5634